# Datawrangling - Capstone 1
***

## Beer Review Data Description
***

The dataset we have contains over 1.5 million reviews of various beers from two websites: BeerAdvocate.com. This data not only includes user reviews, product category and alcohol by volume(ABV), but sensory aspects as well such as taste, look, smell and overall ratings. For this project we will train and test models to predict beer ratings and beer style based of the user reviews that were left.

These reviews were made available by Julian Mcauley, a UCSD Computer Science professor, from a collection period of January 1998 to November 2011. This dataset was accessed with permssion. Here are some key specs of the dataset itself.
+ Number of users: 33,387
+ Number of items: 66,051
+ Number of reviews: 1,586,259
    
To tackle the issue of size, we will take a subset sample of 99,999 reviews to train and test our models before applying it to the rest of the dataset.


### Import and Review Data
***

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from textblob import TextBlob

In [2]:
beersmall = pd.read_excel('data/ratebeer_sample.xlsx')

In [3]:
beersmall.head()

,name,beerId,brewerId,ABV,style,appearance,aroma,palate,taste,overall,time,profileName,text
0,John Harvards Simcoe IPA,63836,8481,5.4,India Pale Ale &#40;IPA&#41;,4,6,3,6,13,1157587200,hopdog,"On tap at the Springfield, PA location. Poured..."
1,John Harvards Simcoe IPA,63836,8481,5.4,India Pale Ale &#40;IPA&#41;,4,6,4,7,13,1157241600,TomDecapolis,On tap at the John Harvards in Springfield PA....
2,John Harvards Cristal Pilsner,71716,8481,5,Bohemian Pilsener,4,5,3,6,14,958694400,PhillyBeer2112,"Springfield, PA. I've never had the Budvar Cri..."
3,John Harvards Fancy Lawnmower Beer,64125,8481,5.4,K•À_lsch,2,4,2,4,8,1157587200,TomDecapolis,On tap the Springfield PA location billed as t...
4,John Harvards Fancy Lawnmower Beer,64125,8481,5.4,K•À_lsch,2,4,2,4,8,1157587200,hopdog,"On tap at the Springfield, PA location. Poured..."


In [4]:
beersmall.columns

Index(['name', 'beerId', 'brewerId', 'ABV', 'style', 'appearance', 'aroma',
       'palate', 'taste', 'overall', 'time', 'profileName', 'text'],
      dtype='object')

In [5]:
beersmall.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 13 columns):
name           99999 non-null object
beerId         99999 non-null object
brewerId       99999 non-null int64
ABV            99999 non-null object
style          99999 non-null object
appearance     99999 non-null int64
aroma          99999 non-null int64
palate         99999 non-null int64
taste          99999 non-null int64
overall        99999 non-null int64
time           99999 non-null int64
profileName    99999 non-null object
text           99999 non-null object
dtypes: int64(7), object(6)
memory usage: 9.9+ MB


## Examining and Cleaning the DataFrame 
***
Examining the first few columns of the dataframe you can notice a few things
+ Special characters and notation weren't read properly. We need to fix this somehow.
+ There is a time column that if we would want to use needs to be altered. In this case we don't need it, so we will drop it.
+ Beers have a range of reviews from 1 to x amount. We want beers to have at least 5 reviews so we need to remove the ones that have less
+ Some beers don't have an alcohol percentage (ABV)
+ Index needs to be redone.
+ The rating system is not on the same scale. "Aroma" and "Taste" ratings are on a scale of 10 while, the rest are on a scale of 5. 
***
### Remove Unecessary Data

In [6]:
# Drop Time Column
beersmall = beersmall.drop(['time'], axis=1)

In [7]:
# Drop any beer styles that have less than 5 reviews
id_count = beersmall.groupby('style')['style'].transform(len)
mask = id_count > 5
beersmall = beersmall[mask]

# Drop any beer names that have less than 5 reviews
id_count = beersmall.groupby('name')['name'].transform(len)
mask = id_count > 5
beersmall = beersmall[mask]

### Fix Spelling Errors

In [8]:
# Check for Beer Styles where the name needs to be altered.
beersmall['style'].unique()

array(['Imperial/Double IPA', 'Sour Ale/Wild Ale', 'Traditional Ale',
       'India Pale Ale &#40;IPA&#41;', 'Brown Ale', 'Porter',
       'Belgian White &#40;Witbier&#41;', 'Imperial Stout', 'Sweet Stout',
       'Stout', 'Scotch Ale', 'Saison', 'American Pale Ale',
       'Belgian Strong Ale', 'Wheat Ale', 'Abt/Quadrupel', 'Premium Lager',
       'Imperial/Strong Porter', 'Pale Lager', 'Fruit Beer',
       'English Pale Ale', 'Spice/Herb/Vegetable', 'Amber Ale', 'Pilsener',
       'Premium Bitter/ESB', 'Vienna', 'Barley Wine', 'Doppelbock',
       'Altbier', 'Abbey Tripel', 'Classic German Pilsener', 'Weizen Bock',
       'Dunkelweizen', 'Oktoberfest/M•À_rzen', 'Dortmunder/Helles',
       'Zwickel/Keller/Landbier', 'German Hefeweizen', 'Sak•À_ - Junmai',
       'Sak•À_ - Nigori', 'Sak•À_ - Daiginjo',
       'Strong Pale Lager/Imperial Pils', 'Dunkel', 'Heller Bock',
       'Golden Ale/Blond Ale', 'Scottish Ale', 'Mild Ale', 'Irish Ale',
       'California Common', 'Dunkler Bock', 'Lo

In [9]:
# Function to replace character errors in the style column
def replacestyle(col):
    # Replace each value that matches the left side of the pair
    return col.replace({
        'K•À_lsch': 'Kölsch',
        '&#40;' : '(',
        '&#41;' : ')',
        'M•À_rzen' : 'Märzen',
        'Sak•À_' : 'Sake',
        'Bi•À_re de Garde' : 'Bière de Garde'
    }, regex=True)

# Apply the function replacestyle to the beer['style'] column
beersmall['style'] = replacestyle(beersmall['style'])

# Function to replace character errors in the name column 
def replacename(col):
    # Replace each value that matches the left side of the pair
    return col.replace({
        '&quot;' : '"', 
        '&#40;' : '(',
        '&#41;' : ')',
        'Brï¿½u' : 'Bräu',
        'Kï¿½r' : 'Kür',
        'Mï¿½r' : 'Mär',
        'hï¿½f' : 'häf',
        'lï¿½n' : 'lán',
        'gï¿½u' : 'gäu',
        'rï¿½n' : 'rän',
        'tï¿½c' : 'tüc'
    }, regex= True)

# Apply the function replacename to the beer.name column
beersmall.name = replacename(beersmall.name)

### Adjust Rating Scales
***
In order to have all the ratings on the same scale the following was done:
+ 'aroma' and 'taste' columns were cut in half to bring them down to a rating scale of 5
+ 'overall' rating column was recalculated based off the other 4 ratings given and brought down to scale of 5

In [10]:
beersmall['aroma'] = round(beersmall['aroma'] / 2)
beersmall['taste'] = round(beersmall['taste'] / 2)
beersmall['overall'] = (beersmall['appearance'] + beersmall['aroma'] + beersmall['palate'] + beersmall['taste']) / 4

In [11]:
beersmall.head()

,name,beerId,brewerId,ABV,style,appearance,aroma,palate,taste,overall,profileName,text
15,Barley Island Barrel-Aged Count Hopula,91592,3228,8,Imperial/Double IPA,4,4.0,4,4.0,4.00,JJClark,Handbottled from trade wth Sprinkle. Pours a n...
16,Barley Island Barrel-Aged Count Hopula,91592,3228,8,Imperial/Double IPA,4,4.0,4,4.0,4.00,CaptainCougar,On tap at the Great Taste of the Midwest (8/9/...
17,Barley Island Barrel-Aged Count Hopula,91592,3228,8,Imperial/Double IPA,3,4.0,3,3.0,3.25,StFun,"On tap. Interesting experiment, but I liked re..."
18,Barley Island Barrel-Aged Count Hopula,91592,3228,8,Imperial/Double IPA,4,2.0,4,2.0,3.00,MI2CA,On cask at BI - Aroma of the Hopula Play-Doh h...
19,Barley Island Barrel-Aged Count Hopula,91592,3228,8,Imperial/Double IPA,3,3.0,3,3.0,3.00,vyvvy,GTMW 08 on cask - Pours bronze orang with a mi...


### Break Down and Clean Text Reviews
*** 
Text reviews often times have spelling errors, extraneous words and punctuation that we may not need when analyzing the information. For this reason we breakdown each text review into the following:
+ word count
+ character count
+ average word length
+ stop words
    + words that are generally considered useless when it comes to performing a sentiment analysis
+ number of upper case words
    + this may indicate moments of anger or rage

In [12]:
def avg_word_len(sentence):
    words = str(sentence).split()
    return (sum(len(word) for word in words)/len(words))

# Determine the word count for each text review: word_count
beersmall['word_count'] = beersmall['text'].apply(lambda x: len(str(x).split(" ")))

# Determine the character count for each text review: char_count
beersmall['char_count'] = beersmall['text'].str.len() ## this also includes spaces

# Determine the average word length per text review: avg_word_len
beersmall['avg_word_len'] = beersmall['text'].apply(lambda x: avg_word_len(x))

# Determine the number of stop words per text review: stop_words
stop = stopwords.words('english')
beersmall['stop_words'] = beersmall['text'].apply(lambda x: len([x for x in str(x).split() if x in stop]))

# Determine the number of uppercase words per text review: upper
beersmall['upper'] = beersmall['text'].apply(lambda x: len([x for x in str(x).split() if x.isupper()]))

beersmall[['text','word_count', 'char_count', 'avg_word_len', 'stop_words', 'upper']].head()

,text,word_count,char_count,avg_word_len,stop_words,upper
15,Handbottled from trade wth Sprinkle. Pours a n...,60,346.0,4.783333,18,0
16,On tap at the Great Taste of the Midwest (8/9/...,66,381.0,5.015873,18,0
17,"On tap. Interesting experiment, but I liked re...",64,378.0,4.921875,23,2
18,On cask at BI - Aroma of the Hopula Play-Doh h...,62,318.0,4.431034,22,2
19,GTMW 08 on cask - Pours bronze orang with a mi...,67,349.0,4.564516,22,1


Furthermore we clean the text into a new column "clean_text" where we perform the following actions:
+ change all characters to lowercase
+ Remove any special characters and punctuation
+ Remove any stop words

In [13]:
# Create a new column clean_text where each text review is cleaned to be prepared for sentiment analyses
# Change all characters in each review to lowercase.
beersmall['clean_text'] = beersmall['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))

# Remove any special characters from each review.
beersmall['clean_text'] = beersmall['clean_text'].str.replace('[^\w\s]','')

# Remove any stop words.
beersmall['clean_text'] = beersmall['clean_text'].apply(lambda x: " ".join(x for x in str(x).split() if x not in stop))

beersmall[['text', 'clean_text']].head()

,text,clean_text
15,Handbottled from trade wth Sprinkle. Pours a n...,handbottled trade wth sprinkle pours dark copp...
16,On tap at the Great Taste of the Midwest (8/9/...,tap great taste midwest 8908 pours transparent...
17,"On tap. Interesting experiment, but I liked re...",tap interesting experiment liked regular hopul...
18,On cask at BI - Aroma of the Hopula Play-Doh h...,cask bi hopula playdoh lots oak vanilla bourbo...
19,GTMW 08 on cask - Pours bronze orang with a mi...,gtmw 08 cask pours bronze orang minimal lots v...


### Extract One Beer Style for Training Set
***
By looking at our value counts we determine that within this chunk of the data, it appears that there are the most reviews for India Pale Ales(IPAs) and therefore we will use these reviews as our training set to test our models. In order to do so we must extract the rows from the beer dataframe that only include IPAs. 

In [14]:
beer_ipa = beersmall[beersmall['style'] == 'India Pale Ale (IPA)'] 

In [15]:
beer_ipa.head()

,name,beerId,brewerId,ABV,style,appearance,aroma,palate,taste,overall,profileName,text,word_count,char_count,avg_word_len,stop_words,upper,clean_text
114,Barley Island Barfly IPA,58511,3228,6.5,India Pale Ale (IPA),3,4.0,4,4.0,3.75,adamlangolf,Another quality session IPA. The lowest ABV o...,82,423.0,4.441558,22,6,another quality session ipa lowest abv ipa wou...
115,Barley Island Barfly IPA,58511,3228,6.5,India Pale Ale (IPA),3,3.0,3,4.0,3.25,GG,"Received in a trade with mmm///beer, so thanks...",117,614.0,4.389381,45,0,received trade mmmbeer thanks bro quite actual...
116,Barley Island Barfly IPA,58511,3228,6.5,India Pale Ale (IPA),3,4.0,4,4.0,3.75,mar,bottle thanks to mmm///beer. deep ruby color ...,33,175.0,4.766667,7,0,thanks mmmbeer deep ruby color 1 finger cream ...
117,Barley Island Barfly IPA,58511,3228,6.5,India Pale Ale (IPA),4,4.0,3,4.0,3.75,jcwattsrugger,on tap-pours a light tan head that laces and a...,59,377.0,6.000000,15,0,tappours tan laces amber color northwest hopsg...
118,Barley Island Barfly IPA,58511,3228,6.5,India Pale Ale (IPA),3,3.0,3,3.0,3.00,Snojerk321,Bottle via trade with mmm///beer. This brew p...,81,475.0,4.937500,24,0,via trade mmmbeer brew pours thick hazy orange...


We have 7818 reviews for 101 different types of IPAs. We can increase this size by incorporating the rest of the reviews at a later time.

## Conclusion
***
To wrap up data wrangling we did the following to the dataset:
+ Took a subset of the entire data collected
+ Dropped specific beers and beer styles that had less than 5 reviews
+ Fixed spelling errors
+ Broke down text reviews into separate columns by words, characters, etc. for further analysis.
+ Cleaned each text review in preparation for sentiment analysis
+ Extracted one beer style, IPAs, as our training dataset.


## Sources
***
Lipton, Zachary & Vikram, Sharad & McAuley, Julian. (2015). Capturing Meaning in Product Reviews with Character-Level Generative Text Models.
https://www.researchgate.net/publication/283761921_Capturing_Meaning_in_Product_Reviews_with_Character-Level_Generative_Text_Models